In [1]:
import time
import pprint
import numpy as np
import DRL.takeoff_dqn as tfdqn
import LLCsimple
import LLC 
import scaffold.fgdata as dfer
import scaffold.pidpilot as PID
import datetime
import fgmodule.fgenv as fgenv
import pandas as pd
import os
from scaffold.timer import gettime
import gym

'''
##########controlframe############


##########state_dict##############
{'aileron': 0.0, 'elevator': 0.0, 'rudder': -0.026, 'flaps': 0.0, 'throttle0': 0.6, 'throttle1': 0.6, 'vsi-fpm': 0.0, 'alt-ft': -372.808502, 'ai-pitch': 0.401045, 'ai-roll': 0.050598, 'ai-offset': 0.0, 'hi-heading': 80.568947, 'roll-deg': 0.050616, 'pitch-deg': 0.401055, 'heading-deg': 90.013458, 'airspeed-kt': 71.631187, 'speed-north-fps': -0.021637, 'speed-east-fps': 119.609383, 'speed-down-fps': -0.071344, 'uBody-fps': 119.606964, 'vBody-fps': -0.005778, 'wBody-fps': 0.765776, 'north-accel-fps_sec': 0.118887, 'east-accel-fps_sec': 5.870498, 'down-accel-fps_sec': -0.003219, 'x-accel-fps_sec': 6.095403, 'y-accel-fps_sec': -0.148636, 'z-accel-fps_sec': -32.113453, 'latitude': 21.325245, 'longitude': -157.93947, 'altitude': 22.297876, 'crashed': 0.0}
'''

LLC_FEATURE_BOUNDS = {
    'aileron': [-1, 1],  # 副翼 控制飞机翻滚 [-1,1] left/right
    'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
    'throttle0': [0, 1],  # 油门0
    'pitch-deg': [-90., 90.],  # 飞机俯仰角
    'roll-deg': [-180., 180.],  # 飞机滚转角
    'heading-deg': [0., 360.],  # 飞机朝向
}
LLC_GOAL_BOUNDS = {
    'pitch-deg': [-90., 90.],  # 飞机俯仰角
    'roll-deg': [-180., 180.],  # 飞机滚转角
    'heading-deg': [0., 360.],  # 飞机朝向
}
goals = {
    'pitch-deg': 0.,  # 飞机俯仰角
    'roll-deg': 0.,  # 飞机滚转角
    'heading-deg': 90.,  # 飞机朝向
}

LLC_ACTION_BOUNDS = {
    'aileron': [-1, 1],  # 副翼 控制飞机翻滚 [-1,1] left/right
    'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
    'throttle': [0, 1],  # 油门0
}

def fgstart(fg2client_addr=("127.0.0.1", 5700), client2fg_addr=("127.0.0.1", 5701), telnet_addr=("127.0.0.1", 5555)):
    '''
    简化FG通信启动过程
    ---
    Inputs:
        - fg2client_addr
        - client2fg_addr
        - telnet_addr
    Outputs:
        - myfgenv:初始话完成的fgenv类

    '''
    myfgenv = fgenv.fgenv(telnet_addr, fg2client_addr, client2fg_addr)
    # initial_state = myfgenv.initial()
    myfgenv.initial()

    return myfgenv


MAX_EPISODE = 1000
MAX_EP_STEPS = 200

GAMMA = 0.9
LR_A = 0.001    # learning rate for actor
LR_C = 0.01     # learning rate for critic

##
epoch = 100
step = 300

In [71]:
myfgenv = fgstart()

# bounds = {
#     'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
# }

flylog saving path check pass!
send data to 127.0.0.1:5701 !
fgudp is ready!!!
Bind UDP on 127.0.0.1:5700 !
save log to data/flylog
auto start完成 13 : 45 : 52
fgudp stop begin! waiting for 2 thread quit
send data to 127.0.0.1:5701 stop!
UDP rece on 127.0.0.1:5700 stop!
reset完成 13 : 45 : 53
flylog saving path check pass!
send data to 127.0.0.1:5701 !
fgudp is ready!!!
Bind UDP on 127.0.0.1:5700 !
save log to data/flylog
auto start完成 13 : 46 : 48


In [2]:
myllc = LLCsimple.LLC(LLC_FEATURE_BOUNDS,LLC_GOAL_BOUNDS,LLC_ACTION_BOUNDS)

In [6]:
state = myfgenv.replay()

replay to ground 完成!time:13 : 28 : 47
auto start完成 13 : 28 : 50


In [3]:
state = {'aileron': 0.0,
 'elevator': 0.0,
 'rudder': 0.0,
 'flaps': 0.0,
 'throttle0': 0.0,
 'throttle1': 0.0,
 'vsi-fpm': 0.0,
 'alt-ft': -370.319122,
 'ai-pitch': 0.534232,
 'ai-roll': 0.277171,
 'ai-offset': 0.0,
 'hi-heading': 83.792358,
 'roll-deg': 0.056131,
 'pitch-deg': 0.467942,
 'heading-deg': 90.00074,
 'airspeed-kt': 1.532065,
 'speed-north-fps': -0.000281,
 'speed-east-fps': -0.007067,
 'speed-down-fps': 0.000105,
 'uBody-fps': -0.007067,
 'vBody-fps': 0.000281,
 'wBody-fps': 4.7e-05,
 'north-accel-fps_sec': -1.4e-05,
 'east-accel-fps_sec': -0.009769,
 'down-accel-fps_sec': -0.000142,
 'x-accel-fps_sec': 0.252822,
 'y-accel-fps_sec': -0.031483,
 'z-accel-fps_sec': -32.151367,
 'latitude': 21.325247,
 'longitude': -157.943137,
 'altitude': 21.358186,
 'crashed': 0.0}

In [4]:
goal = goals

In [5]:
myllc.goals

{'pitch-deg': [-90.0, 90.0],
 'roll-deg': [-180.0, 180.0],
 'heading-deg': [0.0, 360.0]}

array([-0.30626116,  0.13050586, -0.09737893])

In [69]:
action ,_ = myllc.choose_action(state,goal)
print(LLCsimple.llc_reward(state , goal, old_action, action, 0))
print(action)
print(old_action)
old_action = action

-0.35585293870852
[0.22711391 0.07321904 0.14892279]
[-0.14034061 -0.37562894  0.03714219]


In [3]:
import importlib
importlib.reload(LLCsimple)

<module 'LLCsimple' from 'E:\\project\\flightgear\\FGAutopilot\\LLCsimple.py'>

In [70]:
action = np.array([0.,0.,0.])
old_action = np.array([0.,0.,0.])

In [ ]:
 for e in range(epoch):
        state = myfgenv.replay()
        time.sleep(2)

        next_goal = goal

        ep_reward = 0

        for s in range(step):
        
            
            old_action = action 
            action,action_true = myllc.choose_action(state,goal)
            

            action_frame = dfer.action2frame((action_true[0],0.0,action_true[1],action_true[2],action_true[2]))
            
            next_state, reward , done , info = myfgenv.step(action_frame) 
            
            r_ = LLCsimple.llc_reward(state , goal,old_action,action ,reward)
            
            next_goal = goals
            myllc.learn(state, goal, r_, action,next_state , next_goal)
            
            state = next_state
            goal = next_goal

            ep_reward += r_
            if done:
                print('Episode:', e, ' Reward: %i' %
                      int(ep_reward), 'Explore: %.2f' % myllc.var, )
                break
            
            if s == step-1:
                print('Episode:', e, ' Reward: %i' %
                      int(ep_reward), 'Explore: %.2f' % myllc.var, )
                # if ep_reward > -300:RENDER = True
                break
            time.sleep(0.1)

replay to ground 完成!time:13 : 47 : 21
auto start完成 13 : 47 : 24
Episode: 0  Reward: -197 Explore: 0.30
replay to ground 完成!time:13 : 47 : 56
auto start完成 13 : 47 : 59
Episode: 1  Reward: -235 Explore: 0.30
replay to ground 完成!time:13 : 48 : 35
auto start完成 13 : 48 : 38
Episode: 2  Reward: -184 Explore: 0.30
replay to ground 完成!time:13 : 49 : 6
auto start完成 13 : 49 : 9
Episode: 3  Reward: -243 Explore: 0.30
replay to ground 完成!time:13 : 49 : 45
auto start完成 13 : 49 : 48
Episode: 4  Reward: -215 Explore: 0.30
replay to ground 完成!time:13 : 50 : 24
auto start完成 13 : 50 : 27
Episode: 5  Reward: -223 Explore: 0.30
replay to ground 完成!time:13 : 51 : 3
auto start完成 13 : 51 : 6
Episode: 6  Reward: -235 Explore: 0.30
replay to ground 完成!time:13 : 51 : 42
auto start完成 13 : 51 : 45
Episode: 7  Reward: -234 Explore: 0.30
replay to ground 完成!time:13 : 52 : 21
auto start完成 13 : 52 : 24
Episode: 8  Reward: -242 Explore: 0.30
replay to ground 完成!time:13 : 52 : 59
auto start完成 13 : 53 : 2
Episode: 9  Re

In [38]:
myllc.ddpg.pointer

25081

In [37]:
t1= time.clock()
myllc.learn(state, goal, r_, action,next_state , next_goal)
print(time.clock()-t1)

0.0027973283603017762
